In [2]:
import pandas as pd
import numpy as np
import os
import re # Modul untuk operasi ekspresi reguler

# drop provinsi
values_to_drop = [
    'Semua Provinsi', 'Bali', 'Nusa Tenggara Barat', 'Nusa Tenggara Timur',
    'Gorontalo', 'Sulawesi Selatan', 'Sulawesi Tenggara', 'Sulawesi Tengah',
    'Sulawesi Utara', 'Sulawesi Barat', 'Maluku', 'Maluku Utara',
    'Papua', 'Papua Barat'
]

folder_path = "dataset/mentah"
dataframes = {}

for filename in os.listdir(folder_path):
    if filename.startswith("dataset_") and filename.endswith(".xlsx"):
        file_path = os.path.join(folder_path, filename)
        
        # Pisahkan nama file dari ekstensi (.xlsx)
        base_name_with_prefix = os.path.splitext(filename)[0]
        # [1] mengambil bagian setelah underscore pertama
        core_name = base_name_with_prefix.split('_', 1)[1]
        df_name = "df_" + core_name

        try: 
            df = pd.read_excel(file_path)
            if 'Kabupaten/Kota' in df.columns:
                df['Kabupaten/Kota'] = df['Kabupaten/Kota'].str.strip()
                df = df[~df['Kabupaten/Kota'].isin(values_to_drop)].copy()
                print(f"File '{filename}' dimuat dan difilter. Disimpan sebagai '{df_name}'.")
            else:
                print(f"Peringatan: Kolom 'Kabupaten/Kota' tidak ditemukan di '{filename}'. Hanya dimuat.")

            print(df.head())
            dataframes[df_name] = df

        except Exception as e:
            print(f"Gagal memproses file '{filename}': {e}")

# Membuat variabel global (misalnya df_ayam, df_bawang_merah) dari dictionary
globals().update(dataframes)

File 'dataset_ayam.xlsx' dimuat dan difilter. Disimpan sebagai 'df_ayam'.
      Kabupaten/Kota 01/ 01/ 2020 02/ 01/ 2020 03/ 01/ 2020 06/ 01/ 2020  \
0       Kab. Banggai          NaN          NaN          NaN          NaN   
1     Kab. Bulukumba            -       36,750            -       36,750   
2     Kab. Gorontalo          NaN          NaN          NaN          NaN   
3  Kab. Lombok Timur            -       30,000       30,000       30,000   
4        Kab. Majene            -       30,000       30,000       28,750   

  07/ 01/ 2020 08/ 01/ 2020 09/ 01/ 2020 10/ 01/ 2020 13/ 01/ 2020  ...  \
0          NaN          NaN          NaN          NaN          NaN  ...   
1       35,000       34,000       35,000            -       27,250  ...   
2          NaN          NaN          NaN          NaN          NaN  ...   
3       36,000       36,500       36,500       38,000       38,000  ...   
4       28,750       27,500       27,500       27,500       27,500  ...   

  18/ 12/ 2024 19/

In [7]:
import pandas as pd
import numpy as np
import os
import re

PERCENT_THRESHOLD = 0.4 

output_folder = "dataset/filled"
os.makedirs(output_folder, exist_ok=True)

for name, df in dataframes.items():
    print(f"\n=== Processing {name.upper()} ===")

    harga_cols = df.columns[1:]
    df[harga_cols] = df[harga_cols].replace({'-': np.nan, '': np.nan})

    # ubah jadi numerik
    for col in harga_cols:
        df[col] = (
            df[col].astype(str)
            .str.replace(',', '', regex=False)
            .pipe(pd.to_numeric, errors='coerce')
        )

    yearly_cols = {}
  
    for col in harga_cols:
        # Pola r'\d{4}' mencari pola 4 digit (YYYY).
        match = re.search(r'\d{4}', col)

        if match:
            year = int(match.group(0)) # Mengambil 4 digit angka yang ditemukan
        
            if year not in yearly_cols:
                yearly_cols[year] = []
            yearly_cols[year].append(col)
        

    print(f"Tahun ditemukan: {list(yearly_cols.keys())}")
    kab_hapus = []

    for idx, row in df.iterrows():
        kab_name = row["Kabupaten/Kota"]
        should_be_removed = False

        for year, cols in yearly_cols.items():
            series = row[cols]
            total_days_in_year = len(cols)

            if total_days_in_year == 0:
                continue

            nan_count = series.isna().sum()
            # print(nan_count)
            percent_nan_annual = nan_count / total_days_in_year

            if percent_nan_annual > PERCENT_THRESHOLD:
                print(f"  → Kab/Kota '{kab_name}' dihapus (Tahun {year}: {percent_nan_annual:.2%} kekosongan)")
                should_be_removed = True
                break # Keluar dari loop tahun jika sudah ditemukan pelanggaran

        if should_be_removed:
            kab_hapus.append(kab_name)

    df = df[~df["Kabupaten/Kota"].isin(kab_hapus)].reset_index(drop=True)

    df[harga_cols] = df[harga_cols].fillna(method='ffill', axis=1)
    df[harga_cols] = df[harga_cols].fillna(method='bfill', axis=1)

    output_file = os.path.join(output_folder, f"{name.replace('df_', '')}_filled.xlsx")
    df.to_excel(output_file, index=False)

    print(f"🗑️  {len(kab_hapus)} kabupaten dihapus dari {name}.")
    if kab_hapus:
        print("→", kab_hapus)

    dataframes[name] = df  # update dict-nya juga


=== Processing DF_AYAM ===
Tahun ditemukan: [2020, 2021, 2022, 2023, 2024]
  → Kab/Kota 'Kab. Banggai' dihapus (Tahun 2020: 100.00% kekosongan)
  → Kab/Kota 'Kab. Gorontalo' dihapus (Tahun 2020: 100.00% kekosongan)
  → Kab/Kota 'Kab. Mimika' dihapus (Tahun 2020: 97.33% kekosongan)
  → Kab/Kota 'Kab. Sumba Timur' dihapus (Tahun 2020: 100.00% kekosongan)
  → Kab/Kota 'Kota Kotamobagu' dihapus (Tahun 2020: 100.00% kekosongan)


C:\Users\Gaby\AppData\Local\Temp\ipykernel_17140\1757628579.py:67: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[harga_cols] = df[harga_cols].fillna(method='ffill', axis=1)
C:\Users\Gaby\AppData\Local\Temp\ipykernel_17140\1757628579.py:68: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[harga_cols] = df[harga_cols].fillna(method='bfill', axis=1)


🗑️  5 kabupaten dihapus dari df_ayam.
→ ['Kab. Banggai', 'Kab. Gorontalo', 'Kab. Mimika', 'Kab. Sumba Timur', 'Kota Kotamobagu']

=== Processing DF_BAWANG_MERAH ===
Tahun ditemukan: [2020, 2021, 2022, 2023, 2024]
  → Kab/Kota 'Kab. Banggai' dihapus (Tahun 2020: 100.00% kekosongan)
  → Kab/Kota 'Kab. Gorontalo' dihapus (Tahun 2020: 100.00% kekosongan)
  → Kab/Kota 'Kab. Mimika' dihapus (Tahun 2020: 97.33% kekosongan)
  → Kab/Kota 'Kab. Sumba Timur' dihapus (Tahun 2020: 100.00% kekosongan)
  → Kab/Kota 'Kota Kotamobagu' dihapus (Tahun 2020: 100.00% kekosongan)


C:\Users\Gaby\AppData\Local\Temp\ipykernel_17140\1757628579.py:67: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[harga_cols] = df[harga_cols].fillna(method='ffill', axis=1)
C:\Users\Gaby\AppData\Local\Temp\ipykernel_17140\1757628579.py:68: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[harga_cols] = df[harga_cols].fillna(method='bfill', axis=1)


🗑️  5 kabupaten dihapus dari df_bawang_merah.
→ ['Kab. Banggai', 'Kab. Gorontalo', 'Kab. Mimika', 'Kab. Sumba Timur', 'Kota Kotamobagu']

=== Processing DF_BAWANG_PUTIH ===
Tahun ditemukan: [2020, 2021, 2022, 2023, 2024]
  → Kab/Kota 'Kab. Banggai' dihapus (Tahun 2020: 100.00% kekosongan)
  → Kab/Kota 'Kab. Gorontalo' dihapus (Tahun 2020: 100.00% kekosongan)
  → Kab/Kota 'Kab. Mimika' dihapus (Tahun 2020: 97.33% kekosongan)
  → Kab/Kota 'Kab. Sumba Timur' dihapus (Tahun 2020: 100.00% kekosongan)
  → Kab/Kota 'Kota Kotamobagu' dihapus (Tahun 2020: 100.00% kekosongan)


...

🗑️  5 kabupaten dihapus dari df_minyak.
→ ['Kab. Banggai', 'Kab. Gorontalo', 'Kab. Mimika', 'Kab. Sumba Timur', 'Kota Kotamobagu']

=== Processing DF_SAPI ===
Tahun ditemukan: [2020, 2021, 2022, 2023, 2024]
  → Kab/Kota 'Kab. Banggai' dihapus (Tahun 2020: 100.00% kekosongan)
  → Kab/Kota 'Kab. Gorontalo' dihapus (Tahun 2020: 100.00% kekosongan)
  → Kab/Kota 'Kab. Mimika' dihapus (Tahun 2020: 97.33% kekosongan)
  → Kab/Kota 'Kab. Sumba Timur' dihapus (Tahun 2020: 100.00% kekosongan)
  → Kab/Kota 'Kota Kotamobagu' dihapus (Tahun 2020: 100.00% kekosongan)


C:\Users\Gaby\AppData\Local\Temp\ipykernel_17140\1757628579.py:67: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[harga_cols] = df[harga_cols].fillna(method='ffill', axis=1)
C:\Users\Gaby\AppData\Local\Temp\ipykernel_17140\1757628579.py:68: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[harga_cols] = df[harga_cols].fillna(method='bfill', axis=1)


🗑️  5 kabupaten dihapus dari df_sapi.
→ ['Kab. Banggai', 'Kab. Gorontalo', 'Kab. Mimika', 'Kab. Sumba Timur', 'Kota Kotamobagu']

=== Processing DF_TELUR ===
Tahun ditemukan: [2020, 2021, 2022, 2023, 2024]
  → Kab/Kota 'Kab. Banggai' dihapus (Tahun 2020: 100.00% kekosongan)
  → Kab/Kota 'Kab. Gorontalo' dihapus (Tahun 2020: 100.00% kekosongan)
  → Kab/Kota 'Kab. Mimika' dihapus (Tahun 2020: 97.33% kekosongan)
  → Kab/Kota 'Kab. Sumba Timur' dihapus (Tahun 2020: 100.00% kekosongan)
  → Kab/Kota 'Kota Kotamobagu' dihapus (Tahun 2020: 100.00% kekosongan)


C:\Users\Gaby\AppData\Local\Temp\ipykernel_17140\1757628579.py:67: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[harga_cols] = df[harga_cols].fillna(method='ffill', axis=1)
C:\Users\Gaby\AppData\Local\Temp\ipykernel_17140\1757628579.py:68: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[harga_cols] = df[harga_cols].fillna(method='bfill', axis=1)


🗑️  5 kabupaten dihapus dari df_telur.
→ ['Kab. Banggai', 'Kab. Gorontalo', 'Kab. Mimika', 'Kab. Sumba Timur', 'Kota Kotamobagu']


In [8]:
import pandas as pd
import os

folder_path = 'dataset/filled'  # Ganti dengan path folder Anda
output_file = 'dataset/filled/semua_komoditas_filled.xlsx'

excel_files = [f for f in os.listdir(folder_path) if f.endswith(('.xlsx'))]
writer = pd.ExcelWriter(output_file, engine='xlsxwriter')

for filename in excel_files:
    file_path = os.path.join(folder_path, filename)

    base_name = os.path.splitext(filename)[0]
    sheet_name = base_name.removesuffix('_filled')

    try:
        # 4c. Baca data dari file Excel
        df = pd.read_excel(file_path)
        print(df.dtypes)
        
        # 4d. Tulis DataFrame ke sheet baru di file output
        df.to_excel(writer, sheet_name=sheet_name, index=False)
        print(f"  ✅ Data dari '{filename}' berhasil ditambahkan ke sheet '{sheet_name}'")
        
    except Exception as e:
        print(f"  ❌ Gagal memproses file {filename}: {e}")


writer.close() 
print(f"File gabungan telah dibuat: {output_file}")

Kabupaten/Kota    object
01/ 01/ 2020       int64
02/ 01/ 2020       int64
03/ 01/ 2020       int64
06/ 01/ 2020       int64
                   ...  
25/ 12/ 2024       int64
26/ 12/ 2024       int64
27/ 12/ 2024       int64
30/ 12/ 2024       int64
31/ 12/ 2024       int64
Length: 1306, dtype: object
  ✅ Data dari 'ayam_filled.xlsx' berhasil ditambahkan ke sheet 'ayam'
Kabupaten/Kota    object
01/ 01/ 2020       int64
02/ 01/ 2020       int64
03/ 01/ 2020       int64
06/ 01/ 2020       int64
                   ...  
25/ 12/ 2024       int64
26/ 12/ 2024       int64
27/ 12/ 2024       int64
30/ 12/ 2024       int64
31/ 12/ 2024       int64
Length: 1306, dtype: object
  ✅ Data dari 'bawang_merah_filled.xlsx' berhasil ditambahkan ke sheet 'bawang_merah'
Kabupaten/Kota    object
01/ 01/ 2020       int64
02/ 01/ 2020       int64
03/ 01/ 2020       int64
06/ 01/ 2020       int64
                   ...  
25/ 12/ 2024       int64
26/ 12/ 2024       int64
27/ 12/ 2024       int64
30/ 12/ 2024 

In [11]:
import pandas as pd
import numpy as np

input_path = "dataset/filled/semua_komoditas_filled.xlsx"  # ganti dengan path file kamu
sheets = pd.read_excel(input_path, sheet_name=None)

monthly_price = {}

for sheet_name, df in sheets.items():
    kabupaten = df.iloc[:, 0]             # kolom pertama = kabupaten/kota
    numeric_df = df.iloc[:, 1:]           # sisanya = data angka

    # --- Bersihin kolom tanggal ---
    cleaned_columns = (
        numeric_df.columns
        .astype(str)
        .str.strip()
        .str.replace(r"\s+", "", regex=True)      # hapus semua spasi
        .str.replace(r"[^0-9/]", "", regex=True)  # buang karakter non angka dan non "/"
    )

    # Coba ubah nama kolom (tanggal) jadi datetime
    try:
        numeric_df.columns = pd.to_datetime(cleaned_columns, format='%d/%m/%Y', errors="coerce")

        monthly_avg = (
            numeric_df
            .T.resample("M").mean()  # 'M' = akhir bulan
            .T
        )

        monthly_avg.columns = [
            f"{sheet_name}_{col.strftime('%b%Y')}" for col in monthly_avg.columns
        ]

        # Tambahkan kolom kabupaten/kota lagi
        monthly_avg.insert(0, "Kabupaten/Kota", kabupaten.values)
        monthly_price[sheet_name] = monthly_avg

        print(f"✅ Sukses menghitung rata-rata bulanan untuk {sheet_name}")
    except:
        print(f"⚠️ Gagal proses di {sheet_name}")

output_path = "dataset/hasil_preprocessed.xlsx"
with pd.ExcelWriter(output_path, engine='openpyxl') as writer:
    for sheet_name, result_df in monthly_price.items():
        result_df.to_excel(writer, sheet_name=sheet_name, index=False)

print(f"✅ Selesai! File disimpan di: {output_path}")



C:\Users\Gaby\AppData\Local\Temp\ipykernel_17140\995447044.py:28: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  .T.resample("M").mean()  # 'M' = akhir bulan
C:\Users\Gaby\AppData\Local\Temp\ipykernel_17140\995447044.py:28: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  .T.resample("M").mean()  # 'M' = akhir bulan
C:\Users\Gaby\AppData\Local\Temp\ipykernel_17140\995447044.py:28: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  .T.resample("M").mean()  # 'M' = akhir bulan
C:\Users\Gaby\AppData\Local\Temp\ipykernel_17140\995447044.py:28: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  .T.resample("M").mean()  # 'M' = akhir bulan
C:\Users\Gaby\AppData\Local\Temp\ipykernel_17140\995447044.py:28: FutureWarning: 'M' is deprecated and will be removed in a future version, please use '

✅ Sukses menghitung rata-rata bulanan untuk ayam
✅ Sukses menghitung rata-rata bulanan untuk bawang_merah
✅ Sukses menghitung rata-rata bulanan untuk bawang_putih
✅ Sukses menghitung rata-rata bulanan untuk beras
✅ Sukses menghitung rata-rata bulanan untuk cabai_merah
✅ Sukses menghitung rata-rata bulanan untuk cabai_rawit
✅ Sukses menghitung rata-rata bulanan untuk gula
✅ Sukses menghitung rata-rata bulanan untuk minyak
✅ Sukses menghitung rata-rata bulanan untuk sapi
✅ Sukses menghitung rata-rata bulanan untuk telur
✅ Selesai! File disimpan di: dataset/hasil_preprocessed.xlsx


In [ ]:
import pandas as pd

# --- 1. Baca semua sheet dari file hasil normalisasi ---
input_path = "dataset/hasil_preprocessed.xlsx"
sheets = pd.read_excel(input_path, sheet_name=None)

merged_df = None
for sheet_name, df in sheets.items():
    # df.columns = [col.lower() for col in df.columns]

    temp = df
    # temp.columns = [
    #         "Kabupaten/Kota",
    #         f"{sheet_name.lower()}_mean",
    #         f"{sheet_name.lower()}_cv",
    #         f"{sheet_name.lower()}_slope"
    #     ]
        
    if merged_df is None:
        merged_df = temp
    else:
        merged_df = pd.merge(merged_df, temp, on="Kabupaten/Kota", how="inner")

print("🔹 Data gabungan:", merged_df.shape)
print("🔹 Kolom pada gabungan:", list(merged_df.columns))

merged_df.to_excel("dataset/preprocessed_gabungan_komoditas.xlsx", index=False)

X = merged_df.drop("Kabupaten/Kota", axis=1)

scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

df_normalized = pd.DataFrame(
    X_scaled,
    columns=X.columns 
)

df_normalized.insert(0, 'Kabupaten/Kota', merged_df['Kabupaten/Kota'].values)
df_normalized.to_excel("dataset/hasil_normalisasi.xlsx", index=False)

🔹 Data gabungan: (25, 601)
🔹 Kolom pada gabungan: ['Kabupaten/Kota', 'ayam_Jan2020', 'ayam_Feb2020', 'ayam_Mar2020', 'ayam_Apr2020', 'ayam_May2020', 'ayam_Jun2020', 'ayam_Jul2020', 'ayam_Aug2020', 'ayam_Sep2020', 'ayam_Oct2020', 'ayam_Nov2020', 'ayam_Dec2020', 'ayam_Jan2021', 'ayam_Feb2021', 'ayam_Mar2021', 'ayam_Apr2021', 'ayam_May2021', 'ayam_Jun2021', 'ayam_Jul2021', 'ayam_Aug2021', 'ayam_Sep2021', 'ayam_Oct2021', 'ayam_Nov2021', 'ayam_Dec2021', 'ayam_Jan2022', 'ayam_Feb2022', 'ayam_Mar2022', 'ayam_Apr2022', 'ayam_May2022', 'ayam_Jun2022', 'ayam_Jul2022', 'ayam_Aug2022', 'ayam_Sep2022', 'ayam_Oct2022', 'ayam_Nov2022', 'ayam_Dec2022', 'ayam_Jan2023', 'ayam_Feb2023', 'ayam_Mar2023', 'ayam_Apr2023', 'ayam_May2023', 'ayam_Jun2023', 'ayam_Jul2023', 'ayam_Aug2023', 'ayam_Sep2023', 'ayam_Oct2023', 'ayam_Nov2023', 'ayam_Dec2023', 'ayam_Jan2024', 'ayam_Feb2024', 'ayam_Mar2024', 'ayam_Apr2024', 'ayam_May2024', 'ayam_Jun2024', 'ayam_Jul2024', 'ayam_Aug2024', 'ayam_Sep2024', 'ayam_Oct2024', 'ay

: 